In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pymysql
from datetime import datetime
import requests

In [ ]:
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver.get("https://www.kcar.com/bc/search")

In [ ]:
# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# driver.get("https://www.xn--tv-9z9j31p.com/search/detail/112540867")
# # url3 = 'https://api.kcar.com/bc/search/list/drct'
# # head3 = {'user-agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36"}
# # payload3 = {"enc":"Vbnr653g7BIoeJ3tFO4oXl6hkxgwnnyI5YP06+cEtu5EiS20dXH7OkIACT/Y+trBxTCzuA2a9aiQudzJcJAkcVB1JHrEyBaexlc0TGCb7ZrGgu5plkvNrQ0+BOT3+MGFKfzBjhmfULBKAoFWKoMdebQMYp+7nJROcXStYaYxkVY7vjd38UIfPj3sJFyZOd17YJpynX7PGPN/9Icb2sF4z8TFhBn5eJxF+RxFv66/u3pKWSX0Tf7y06b/P2a/u/jn"}
# p=requests.post(url3) #headers=head3, json=payload3)
# r = p.json()['data']['rows']
# for x in r: 
#     print(x['carCd'])

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
car_id_list = []
for page in range(486): #페이지 반복 국산차만
    url = f'https://www.xn--tv-9z9j31p.com/search/model/kor/{page}?order=&ascending=desc&view=image&customSelect=24&carName=&maker=&model=&dmodel=&grade=&dgrade=&price-min=&price-max=&year-min=&year-max=&usekm-min=&usekm-max=&fuel=&mission=&color=&country=kor&carNo=&carPlateNumber=&vehicle-model=&vehicle-dmodel=&vehicle-name=&tab=model&detailSearch=&type='
    r = requests.get(url) # 현재 페이지 차량ID
    x = BeautifulSoup(r.text).find_all('a', href=True)
    for a in x:
        href = a['href']
        if '/search/detail/' in href:
            car_id = href.split('/search/detail/')[1]
            car_id_list.append(car_id)
car_id_list = sorted(list(set(car_id_list)))
car_id_list
        

['111076127',
 '111077615',
 '111078268',
 '111085372',
 '111091192',
 '111093767',
 '111097389',
 '111100876',
 '111106291',
 '111107397',
 '111107717',
 '111111107',
 '111113178',
 '111115332',
 '111116380',
 '111117231',
 '111117540',
 '111121209',
 '111122544',
 '111124510',
 '111128099',
 '111132478',
 '111134444',
 '111138562',
 '111143509',
 '111143888',
 '111144107',
 '111147941',
 '111152047',
 '111152109',
 '111154349',
 '111154512',
 '111163081',
 '111163922',
 '111163944',
 '111164671',
 '111164747',
 '111166236',
 '111167166',
 '111176034',
 '111179096',
 '111181542',
 '111183272',
 '111184507',
 '111195028',
 '111195639',
 '111195837',
 '111197498',
 '111198796',
 '111199937',
 '111200649',
 '111202320',
 '111202725',
 '111203574',
 '111203717',
 '111206749',
 '111206859',
 '111208243',
 '111213097',
 '111217212',
 '111217725',
 '111220115',
 '111222082',
 '111222250',
 '111223066',
 '111224592',
 '111226256',
 '111227984',
 '111228131',
 '111231162',
 '111250378',
 '1112

In [27]:
len(car_id_list)

11629

In [33]:
k_total_detail = []
k_total_option = []
k_total_seller = []
k_total_store = []
k_total_manu = []
for K_ID in car_id_list:
    kor_car_url = f'https://www.xn--tv-9z9j31p.com/search/detail/{K_ID}'
    kor_r = requests.get(kor_car_url, headers={'User-Agent': 'Mozilla/5.0'}) # 현재 페이지 차량ID
    if BeautifulSoup(kor_r.text).select_one('.basic-info'):
        seller_info = [] #판매자 정보
        seller_data = BeautifulSoup(kor_r.text).find('table', class_="type02").find_all('td')[0:3]
        for idx, seller in enumerate(seller_data):
            if idx == 2:
                seller_info.append(seller.text.split()[0])
            else:
                seller_info.append(seller.text)
        seller_info.append('중카TV')
    
    
        #판매업체 정보
        store = BeautifulSoup(kor_r.text).find('table', class_="type02").find_all('td')[3].text.split('｜')
        store.append("중카TV")
    
    
        #제조사 정보
        x = BeautifulSoup(kor_r.text).find('div', class_="car_name").find('p').text.replace('[', '').replace(']', '/')
        manu = x.split('/')
        if url.split('/')[5] == 'kor':
            manu.append(1)
        else:
            manu.append(0)
    
        car_details = [] #차량 정보
        detial_data = BeautifulSoup(kor_r.text).find('div', class_="basic-info").find_all('td')[0:8]
        price_data = BeautifulSoup(kor_r.text).find('div',class_="car_price").find('span', class_="representativeColor").text
        car_details.append(kor_car_url)
        car_details.append(price_data)
        car_details.append(seller_info[0])
        car_details.append(manu[1])
        for detail in detial_data:
            car_details.append(detail.text)
        del car_details[7]
    
        car_option = []#차량 옵션
        option_url='https://www.xn--tv-9z9j31p.com/search/optionList'
        payload = {'carNo':f'{K_ID}'}
        r = requests.post(option_url, data=payload)
        a = BeautifulSoup(r.text).find_all('ul')
        car_option.append(car_details[9])
        for x in a:
            y = x.find_all('label', class_="representativeColorCheckboxAfter")
            for xx in y:
                car_option.append(xx.text)
        
    
        k_total_detail.append(car_details)
        k_total_option.append(car_option)
        k_total_seller.append(seller_info)
        k_total_store.append(store)
        k_total_manu.append(manu)
        
    
        print(car_details)
        print(car_option)
        print(seller_info)
        print(store)
        print(manu)

['https://www.xn--tv-9z9j31p.com/search/detail/111076127', '450', '안흥도', 'SM7노바 2.5 SE ', '2015', '2015.05.15', '휘발유', '검정색', '209,213km', '11두7578', 'KNMG4D2EMFP021026']
['11두7578', '감지센서 후방', '네비게이션', '블랙박스', '스마트키', '에어컨', '오토도어록', '오토라이트', '전동접이식백미러', '카메라 후방', '크루즈컨트롤 일반', '트립컴퓨터', '파워윈도우', '파워핸들', '핸들리모컨', 'ABS 브레이크 잠김 방지', 'DLR 주간주행등', 'EPB 전자주차브레이크', 'ESP 차체자세제어장치', 'GPS', 'LED리어램프', 'TCS 미끄럼방지', 'TPMS 타이어 공기압센서', '도난경보기', '무선도어잠금장치', '에어백 사이드', '에어백 운전석', '에어백 조수석', '에어백 커튼', '자동도어잠금', '전동사이드미러', '풀오토에어컨', '핸들리모컨', 'AUX 단자', 'AV시스템', 'CD플레이어', 'MP3플레이어', 'USB 단자', '모니터-앞좌석', '블루투스', 'LED램프', '가죽시트', '썬팅', '알루미늄휠', '열선시트 뒷좌석', '열선시트 앞좌석', '전동시트 앞좌석', '패들 시프트']
['안흥도', '010-6888-3771', 'GC21-06445', '중카TV']
['무결점카모터스', '031-215-8484', '중카TV']
['르노코리아(삼성)', 'SM7노바 2.5 SE ', 1]
['https://www.xn--tv-9z9j31p.com/search/detail/111077615', '799', '안흥도', '그랜저 HG LPG HG300 장애인용 ', '2011', '2011.09.30', 'LPG', '은색', '104,211km', '65마9242', 'KMHFH41NBBA087940']
['65마9242', '네비게이션', '스마트키'

ConnectionError: HTTPSConnectionPool(host='www.xn--tv-9z9j31p.com', port=443): Max retries exceeded with url: /search/detail/112284716 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001845AE2C220>: Failed to establish a new connection: [WinError 10060] 연결된 구성원으로부터 응답이 없어 연결하지 못했거나, 호스트로부터 응답이 없어 연결이 끊어졌습니다'))

In [35]:
import csv

In [36]:
with open('k_total_detail.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(k_total_detail)
with open('k_total_option.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(k_total_option)
with open('k_total_seller.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(k_total_seller)
with open('k_total_store.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(k_total_store)
with open('k_total_manu.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerows(k_total_manu)

In [6]:
car_id_list_F = []
for pages_F in range(248): #페이지 반복 수입차차만
    url_F = f'https://www.xn--tv-9z9j31p.com/search/model/foreign/{pages_F}?order=&ascending=desc&view=image&customSelect=24&carName=&maker=&model=&dmodel=&grade=&dgrade=&price-min=&price-max=&year-min=&year-max=&usekm-min=&usekm-max=&fuel=&mission=&color=&country=kor&carNo=&carPlateNumber=&vehicle-model=&vehicle-dmodel=&vehicle-name=&tab=model&detailSearch=&type='
    r_F = requests.get(url_F) # 현재 페이지 차량ID
    x_F= BeautifulSoup(r_F.text).find_all('a', href=True)
    for a_F in x_F:
        href_F = a_F['href']
        if '/search/detail/' in href_F:
            car_id_F = href_F.split('/search/detail/')[1]
            car_id_list_F.append(car_id_F)
car_id_list_F = sorted(list(set(car_id_list_F)))
car_id_list_F

['112365029',
 '112365128',
 '112365182',
 '112365210',
 '112365266',
 '112365284',
 '112365295',
 '112365315',
 '112365353',
 '112365359',
 '112365374',
 '112365426',
 '112365448',
 '112365470',
 '112365476',
 '112365524',
 '112365533',
 '112365608',
 '112365619',
 '112365635',
 '112365683',
 '112365693',
 '112365704',
 '112365719',
 '112365753',
 '112365755',
 '112365759',
 '112365761',
 '112365768',
 '112365788',
 '112365812',
 '112365830',
 '112365831',
 '112365843',
 '112365845',
 '112365853',
 '112365888',
 '112365900',
 '112365911',
 '112365940',
 '112365998',
 '112366004',
 '112366022',
 '112366054',
 '112366061',
 '112366063',
 '112366092',
 '112366120',
 '112366143',
 '112366152',
 '112366211',
 '112366213',
 '112366230',
 '112366262',
 '112366264',
 '112366267',
 '112366269',
 '112366281',
 '112366321',
 '112366340',
 '112366345',
 '112366371',
 '112366382',
 '112366396',
 '112366408',
 '112366411',
 '112366420',
 '112366423',
 '112366425',
 '112366446',
 '112366475',
 '1123

In [ ]:
for F_ID in car_id_list_F:
    Fo_car_url = f'https://www.xn--tv-9z9j31p.com/search/detail/{F_ID}'
    Fo_r = requests.get(Fo_car_url) # 현재 페이지 차량ID


    Fo_seller_info = [] #판매자 정보
    Fo_seller_data = BeautifulSoup(Fo_r.text).find('table', class_="type02").find_all('td')[0:3]
    for Fo_idx, Fo_seller in enumerate(Fo_seller_data):
        if Fo_idx == 2:
            Fo_seller_info.append(Fo_seller.text.split()[0])
        else:
            Fo_seller_info.append(Fo_seller.text)
    Fo_seller_info.append('중카TV')
    


    #판매업체 정보
    Fo_store = BeautifulSoup(Fo_r.text).find('table', class_="type02").find_all('td')[3].text.split('｜')
    Fo_store.append("중카TV")
    


    #제조사 정보
    Fo_x = BeautifulSoup(Fo_r.text).find('div', class_="car_name").find('p').text.replace('[', '').replace(']', '/')
    Fo_manu = x.split('/')
    if url_F.split('/')[5] == 'foreign':
        Fo_manu.append(1)
    else:
        Fo_manu.append(0)
    Fo_manu

    Fo_car_details = [] #차량 정보
    Fo_detial_data = BeautifulSoup(Fo_r.text).find('div', class_="basic-info").find_all('td')[0:8]
    Fo_price_data = BeautifulSoup(Fo_r.text).find('div',class_="car_price").find('span', class_="representativeColor").text
    Fo_car_details.append(Fo_car_url)
    Fo_car_details.append(Fo_price_data)
    Fo_car_details.append(Fo_seller_info[0])
    Fo_car_details.append(Fo_manu[1])
    for Fo_detail in Fo_detial_data:
        Fo_car_details.append(Fo_detail.text)


    Fo_car_option = []#차량 옵션
    Fo_option_url='https://www.xn--tv-9z9j31p.com/search/optionList'
    Fo_payload = {'carNo':f'{F_ID}'}
    Fo_r = requests.post(Fo_option_url, data=Fo_payload)
    Fo_a = BeautifulSoup(Fo_r.text).find_all('ul')
    Fo_car_option.append(Fo_car_details[8])
    for Fo_x in Fo_a:
        Fo_y = Fo_x.find_all('label', class_="representativeColorCheckboxAfter")
        for Fo_xx in Fo_y:
            car_option.append(Fo_xx.text)
    del Fo_car_details[5]



    print(Fo_car_details)
    print(Fo_car_option)
    print(Fo_seller_info)
    print(Fo_store)
    print(Fo_manu)

AttributeError: 'NoneType' object has no attribute 'find_all'